In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('data.csv')
df

,PublishedAgo,Salary,Rating,Employer,JobTitle,Location,Size,Founded,Type,Industry,Sector,Revenue
0,5d,$60K - $80K (Employer est.),4.3,KGS Technology Group Inc\n4.3,Junior Software Engineer,Remote,51 to 200 Employees,2008.0,Company - Private,Information Technology Support Services,Information Technology,$1 to $5 million (USD)
1,4d,$102K - $147K (Employer est.),4.2,"QSC, LLC\n4.2",AI Engineer,Remote,501 to 1000 Employees,1968.0,Company - Private,Electronics Manufacturing,Manufacturing,Unknown / Non-Applicable
2,30d+,$94K - $138K (Glassdoor est.),4.2,MIT Lincoln Laboratory\n4.2,AI Engineer,"Lexington, MA",1001 to 5000 Employees,1951.0,Nonprofit Organization,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable
3,6d,$123K - $283K (Employer est.),4.2,Hewlett Packard Enterprise\n4.2,AI Solution Engineer,"San Jose, CA",10000+ Employees,2015.0,Company - Public,Information Technology Support Services,Information Technology,Unknown / Non-Applicable
4,2d,$140K (Employer est.),3.3,Oran Inc\n3.3,AI/ML Cloud Engineer,Remote,1 to 50 Employees,2004.0,Company - Private,Business Consulting,Management & Consulting,$1 to $5 million (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...
797,9d,$97K - $234K (Employer est.),4.0,PayPal\n4.0,AI Machine Learning Engineer,"San Jose, CA",10000+ Employees,1998.0,Company - Public,Internet & Web Services,Information Technology,$10+ billion (USD)
798,12d,$121K - $163K (Employer est.),3.5,Eightfold.AI\n3.5,Machine Learning Engineer,"Santa Clara, CA",501 to 1000 Employees,2016.0,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable
799,30d+,$121K - $239K (Employer est.),4.4,Adobe\n4.4,Research Scientist/Engineer,"San Jose, CA",10000+ Employees,1982.0,Company - Public,Computer Hardware Development,Information Technology,$5 to $10 billion (USD)
800,30d+,$150K - $200K (Employer est.),4.5,Curai\n4.5,Machine Learning Engineer (Remote),"San Francisco, CA",51 to 200 Employees,2017.0,Company - Private,Hospitals & Health Clinics,Healthcare,Unknown / Non-Applicable


# Cleaning the dataframe data

We are going to go through column by column, to see if it is necessary to clean up the data.
To begin with, let's proceed with the salary column. As you can see, the salary we have extracted comes along with information whether it is an estimate from Glassdoor or from the workers. In addition, there are some cases where the salaries are given by hours, instead of annual salary.

In [3]:
pattern = r'\$(\d+\.?\d*)'

df['SalarySeparated'] = df['Salary'].apply(lambda x: re.findall(pattern, x))

In [4]:
def calculate_min_salary(row):
    salary_list = row['SalarySeparated']
    if 'Per Hour' in row['Salary']:
        return float(salary_list[0]) * 1800 / 1000 if salary_list else -1
    else:
        return float(salary_list[0]) if salary_list else -1

def calculate_max_salary(row):
    salary_list = row['SalarySeparated']
    if 'Per Hour' in row['Salary']:
        return float(salary_list[0]) * 1800 / 1000 if len(salary_list) == 1 else float(salary_list[1]) * 1800 / 1000 if len(salary_list) == 2 else -1
    else:
        return float(salary_list[0]) if len(salary_list) == 1 else float(salary_list[1]) if len(salary_list) == 2 else -1

df['MinSalary'] = df.apply(calculate_min_salary, axis=1)
df['MaxSalary'] = df.apply(calculate_max_salary, axis=1)
df.drop(['Salary', 'SalarySeparated'], axis=1, inplace=True)
df.head(10)

,PublishedAgo,Rating,Employer,JobTitle,Location,Size,Founded,Type,Industry,Sector,Revenue,MinSalary,MaxSalary
0,5d,4.3,KGS Technology Group Inc\n4.3,Junior Software Engineer,Remote,51 to 200 Employees,2008.0,Company - Private,Information Technology Support Services,Information Technology,$1 to $5 million (USD),60.0,80.0
1,4d,4.2,"QSC, LLC\n4.2",AI Engineer,Remote,501 to 1000 Employees,1968.0,Company - Private,Electronics Manufacturing,Manufacturing,Unknown / Non-Applicable,102.0,147.0
2,30d+,4.2,MIT Lincoln Laboratory\n4.2,AI Engineer,"Lexington, MA",1001 to 5000 Employees,1951.0,Nonprofit Organization,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable,94.0,138.0
3,6d,4.2,Hewlett Packard Enterprise\n4.2,AI Solution Engineer,"San Jose, CA",10000+ Employees,2015.0,Company - Public,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,123.0,283.0
4,2d,3.3,Oran Inc\n3.3,AI/ML Cloud Engineer,Remote,1 to 50 Employees,2004.0,Company - Private,Business Consulting,Management & Consulting,$1 to $5 million (USD),140.0,140.0
5,9d,5.0,Accede LLC\n5.0,Computer Vision AI Engineer,Remote,51 to 200 Employees,2019.0,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD),108.0,120.6
6,30d+,4.2,Booz Allen Hamilton\n4.2,AI Prompt Engineer,"Bethesda, MD",10000+ Employees,1914.0,Company - Public,Business Consulting,Management & Consulting,$5 to $10 billion (USD),93.0,212.0
7,10d,2.4,Nooks\n2.4,QA Support Engineer,Remote,1 to 50 Employees,NaN,Company - Private,NaN,NaN,Unknown / Non-Applicable,85.0,140.0
8,30d+,5.0,Arize AI\n5.0,Frontend Engineer,"Berkeley, CA",51 to 200 Employees,2020.0,Company - Private,Software Development,Information Technology,Unknown / Non-Applicable,100.0,185.0
9,27d,4.4,Cisco Systems\n4.4,Senior AI Engineer - 1403502,"Annapolis Junction, MD",10000+ Employees,1984.0,Company - Public,Enterprise Software & Network Solutions,Information Technology,$10+ billion (USD),102.0,156.0


We have obtained, by means of a regular expression, the different price ranges, and we have separated them into two columns. In addition, for the rows where the salary was given by hours, an estimation has been made, multiplying this salary by the hours worked in a year.

In [5]:
pattern = r'(.+)\n.+'

df['Employer'] = df['Employer'].apply(lambda x: re.match(pattern, x).group(1))
df

,PublishedAgo,Rating,Employer,JobTitle,Location,Size,Founded,Type,Industry,Sector,Revenue,MinSalary,MaxSalary
0,5d,4.3,KGS Technology Group Inc,Junior Software Engineer,Remote,51 to 200 Employees,2008.0,Company - Private,Information Technology Support Services,Information Technology,$1 to $5 million (USD),60.0,80.0
1,4d,4.2,"QSC, LLC",AI Engineer,Remote,501 to 1000 Employees,1968.0,Company - Private,Electronics Manufacturing,Manufacturing,Unknown / Non-Applicable,102.0,147.0
2,30d+,4.2,MIT Lincoln Laboratory,AI Engineer,"Lexington, MA",1001 to 5000 Employees,1951.0,Nonprofit Organization,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable,94.0,138.0
3,6d,4.2,Hewlett Packard Enterprise,AI Solution Engineer,"San Jose, CA",10000+ Employees,2015.0,Company - Public,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,123.0,283.0
4,2d,3.3,Oran Inc,AI/ML Cloud Engineer,Remote,1 to 50 Employees,2004.0,Company - Private,Business Consulting,Management & Consulting,$1 to $5 million (USD),140.0,140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,9d,4.0,PayPal,AI Machine Learning Engineer,"San Jose, CA",10000+ Employees,1998.0,Company - Public,Internet & Web Services,Information Technology,$10+ billion (USD),97.0,234.0
798,12d,3.5,Eightfold.AI,Machine Learning Engineer,"Santa Clara, CA",501 to 1000 Employees,2016.0,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,121.0,163.0
799,30d+,4.4,Adobe,Research Scientist/Engineer,"San Jose, CA",10000+ Employees,1982.0,Company - Public,Computer Hardware Development,Information Technology,$5 to $10 billion (USD),121.0,239.0
800,30d+,4.5,Curai,Machine Learning Engineer (Remote),"San Francisco, CA",51 to 200 Employees,2017.0,Company - Private,Hospitals & Health Clinics,Healthcare,Unknown / Non-Applicable,150.0,200.0


We remove the company valuation from the company name.

In [6]:
pattern = r'(.+),(.+)|Remote|(.+)'

df[['City', 'State', 'Country']] = df['Location'].str.extract(pattern)

df['Country'] = np.where(df['City'].isna() & df['State'].isna(), 'Remote', df['Country'])

df['City'] = df['City'].fillna('Remote')
df['State'] = df['State'].fillna('Remote')

df.drop('Location', axis=1, inplace=True)
df

,PublishedAgo,Rating,Employer,JobTitle,Size,Founded,Type,Industry,Sector,Revenue,MinSalary,MaxSalary,City,State,Country
0,5d,4.3,KGS Technology Group Inc,Junior Software Engineer,51 to 200 Employees,2008.0,Company - Private,Information Technology Support Services,Information Technology,$1 to $5 million (USD),60.0,80.0,Remote,Remote,Remote
1,4d,4.2,"QSC, LLC",AI Engineer,501 to 1000 Employees,1968.0,Company - Private,Electronics Manufacturing,Manufacturing,Unknown / Non-Applicable,102.0,147.0,Remote,Remote,Remote
2,30d+,4.2,MIT Lincoln Laboratory,AI Engineer,1001 to 5000 Employees,1951.0,Nonprofit Organization,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable,94.0,138.0,Lexington,MA,NaN
3,6d,4.2,Hewlett Packard Enterprise,AI Solution Engineer,10000+ Employees,2015.0,Company - Public,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,123.0,283.0,San Jose,CA,NaN
4,2d,3.3,Oran Inc,AI/ML Cloud Engineer,1 to 50 Employees,2004.0,Company - Private,Business Consulting,Management & Consulting,$1 to $5 million (USD),140.0,140.0,Remote,Remote,Remote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,9d,4.0,PayPal,AI Machine Learning Engineer,10000+ Employees,1998.0,Company - Public,Internet & Web Services,Information Technology,$10+ billion (USD),97.0,234.0,San Jose,CA,NaN
798,12d,3.5,Eightfold.AI,Machine Learning Engineer,501 to 1000 Employees,2016.0,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,121.0,163.0,Santa Clara,CA,NaN
799,30d+,4.4,Adobe,Research Scientist/Engineer,10000+ Employees,1982.0,Company - Public,Computer Hardware Development,Information Technology,$5 to $10 billion (USD),121.0,239.0,San Jose,CA,NaN
800,30d+,4.5,Curai,Machine Learning Engineer (Remote),51 to 200 Employees,2017.0,Company - Private,Hospitals & Health Clinics,Healthcare,Unknown / Non-Applicable,150.0,200.0,San Francisco,CA,NaN


In the 'Location' column we can have three possible combinations: ('city', 'state'), (Remote) or ('country'). To try to separate this information and make it easier to access, we are going to separate it into three different columns.

In [7]:
from geopy.geocoders import Nominatim

def get_country(city, state):
    geolocator = Nominatim(user_agent="my_app")
    location = f"{city}, {state}"

    if location == 'Remote, Remote':
        return 'Remote'
    else:
        location = geolocator.geocode(location)
        if location is not None:
            country = location.raw.get('display_name', '').split(',')[-1].strip()
            return country
        else:
            return
        
df['Country'] = df.apply(lambda row: get_country(row['City'], row['State']), axis=1)
df

,PublishedAgo,Rating,Employer,JobTitle,Size,Founded,Type,Industry,Sector,Revenue,MinSalary,MaxSalary,City,State,Country
0,5d,4.3,KGS Technology Group Inc,Junior Software Engineer,51 to 200 Employees,2008.0,Company - Private,Information Technology Support Services,Information Technology,$1 to $5 million (USD),60.0,80.0,Remote,Remote,Remote
1,4d,4.2,"QSC, LLC",AI Engineer,501 to 1000 Employees,1968.0,Company - Private,Electronics Manufacturing,Manufacturing,Unknown / Non-Applicable,102.0,147.0,Remote,Remote,Remote
2,30d+,4.2,MIT Lincoln Laboratory,AI Engineer,1001 to 5000 Employees,1951.0,Nonprofit Organization,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable,94.0,138.0,Lexington,MA,United States
3,6d,4.2,Hewlett Packard Enterprise,AI Solution Engineer,10000+ Employees,2015.0,Company - Public,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,123.0,283.0,San Jose,CA,United States
4,2d,3.3,Oran Inc,AI/ML Cloud Engineer,1 to 50 Employees,2004.0,Company - Private,Business Consulting,Management & Consulting,$1 to $5 million (USD),140.0,140.0,Remote,Remote,Remote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,9d,4.0,PayPal,AI Machine Learning Engineer,10000+ Employees,1998.0,Company - Public,Internet & Web Services,Information Technology,$10+ billion (USD),97.0,234.0,San Jose,CA,United States
798,12d,3.5,Eightfold.AI,Machine Learning Engineer,501 to 1000 Employees,2016.0,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,121.0,163.0,Santa Clara,CA,United States
799,30d+,4.4,Adobe,Research Scientist/Engineer,10000+ Employees,1982.0,Company - Public,Computer Hardware Development,Information Technology,$5 to $10 billion (USD),121.0,239.0,San Jose,CA,United States
800,30d+,4.5,Curai,Machine Learning Engineer (Remote),51 to 200 Employees,2017.0,Company - Private,Hospitals & Health Clinics,Healthcare,Unknown / Non-Applicable,150.0,200.0,San Francisco,CA,United States


In addition, we will use the geopy library to get the country in cases where we only have the city and state.

In [8]:
def split_size(size):
    if size == '10000+ Employees':
        min_size = 10000
        max_size = 10000  
    elif size == 'Unknown':
        min_size = np.nan
        max_size = np.nan
    else:
        values = size.split(' ')
        min_size = int(values[0])
        max_size = int(values[2])
    return min_size, max_size

df[['MinSize', 'MaxSize']] = df['Size'].apply(lambda x: pd.Series(split_size(x)))
df.drop('Size', axis=1, inplace=True)
df

,PublishedAgo,Rating,Employer,JobTitle,Founded,Type,Industry,Sector,Revenue,MinSalary,MaxSalary,City,State,Country,MinSize,MaxSize
0,5d,4.3,KGS Technology Group Inc,Junior Software Engineer,2008.0,Company - Private,Information Technology Support Services,Information Technology,$1 to $5 million (USD),60.0,80.0,Remote,Remote,Remote,51.0,200.0
1,4d,4.2,"QSC, LLC",AI Engineer,1968.0,Company - Private,Electronics Manufacturing,Manufacturing,Unknown / Non-Applicable,102.0,147.0,Remote,Remote,Remote,501.0,1000.0
2,30d+,4.2,MIT Lincoln Laboratory,AI Engineer,1951.0,Nonprofit Organization,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable,94.0,138.0,Lexington,MA,United States,1001.0,5000.0
3,6d,4.2,Hewlett Packard Enterprise,AI Solution Engineer,2015.0,Company - Public,Information Technology Support Services,Information Technology,Unknown / Non-Applicable,123.0,283.0,San Jose,CA,United States,10000.0,10000.0
4,2d,3.3,Oran Inc,AI/ML Cloud Engineer,2004.0,Company - Private,Business Consulting,Management & Consulting,$1 to $5 million (USD),140.0,140.0,Remote,Remote,Remote,1.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,9d,4.0,PayPal,AI Machine Learning Engineer,1998.0,Company - Public,Internet & Web Services,Information Technology,$10+ billion (USD),97.0,234.0,San Jose,CA,United States,10000.0,10000.0
798,12d,3.5,Eightfold.AI,Machine Learning Engineer,2016.0,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,121.0,163.0,Santa Clara,CA,United States,501.0,1000.0
799,30d+,4.4,Adobe,Research Scientist/Engineer,1982.0,Company - Public,Computer Hardware Development,Information Technology,$5 to $10 billion (USD),121.0,239.0,San Jose,CA,United States,10000.0,10000.0
800,30d+,4.5,Curai,Machine Learning Engineer (Remote),2017.0,Company - Private,Hospitals & Health Clinics,Healthcare,Unknown / Non-Applicable,150.0,200.0,San Francisco,CA,United States,51.0,200.0


We divide the size of the company into minimum size and maximum size.

In [9]:
def min_max_revenue(row):
    row = row.replace('$', '').replace(' (USD)', '').split(' ')
    if row[0] == 'Unknown':
        return np.NaN, np.NaN
    else:
        if row[0][-1] == '+':
            return (int(row[0][:-1]) * 1e6, int(row[0][:-1]) * 1e6) if row[1] == 'million' else (int(row[0][:-1]) * 1e9, int(row[0][:-1]) * 1e9)
        elif row[1] == 'to':
            return (int(row[0]) * 1e6, int(row[2]) * 1e6) if row[3] == 'million' else (int(row[0]) * 1e9, int(row[2]) * 1e9)
        elif row[1] == 'million':
            return (int(row[0]) * 1e6, int(row[3]) * 1e6) if row[4] == 'million' else (int(row[0]) * 1e6, int(row[3]) * 1e9)
        else:
            return (int(row[0]) * 1e9, int(row[3]) * 1e6) if row[4] == 'million' else (int(row[0]) * 1e9, int(row[3]) * 1e9)
            
    
df[['MinRevenue', 'MaxRevenue']] = df['Revenue'].apply(lambda x: pd.Series(min_max_revenue(x)))
df.drop('Revenue', axis=1, inplace=True)
df

,PublishedAgo,Rating,Employer,JobTitle,Founded,Type,Industry,Sector,MinSalary,MaxSalary,City,State,Country,MinSize,MaxSize,MinRevenue,MaxRevenue
0,5d,4.3,KGS Technology Group Inc,Junior Software Engineer,2008.0,Company - Private,Information Technology Support Services,Information Technology,60.0,80.0,Remote,Remote,Remote,51.0,200.0,1.000000e+06,5.000000e+06
1,4d,4.2,"QSC, LLC",AI Engineer,1968.0,Company - Private,Electronics Manufacturing,Manufacturing,102.0,147.0,Remote,Remote,Remote,501.0,1000.0,NaN,NaN
2,30d+,4.2,MIT Lincoln Laboratory,AI Engineer,1951.0,Nonprofit Organization,Aerospace & Defense,Aerospace & Defense,94.0,138.0,Lexington,MA,United States,1001.0,5000.0,NaN,NaN
3,6d,4.2,Hewlett Packard Enterprise,AI Solution Engineer,2015.0,Company - Public,Information Technology Support Services,Information Technology,123.0,283.0,San Jose,CA,United States,10000.0,10000.0,NaN,NaN
4,2d,3.3,Oran Inc,AI/ML Cloud Engineer,2004.0,Company - Private,Business Consulting,Management & Consulting,140.0,140.0,Remote,Remote,Remote,1.0,50.0,1.000000e+06,5.000000e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,9d,4.0,PayPal,AI Machine Learning Engineer,1998.0,Company - Public,Internet & Web Services,Information Technology,97.0,234.0,San Jose,CA,United States,10000.0,10000.0,1.000000e+10,1.000000e+10
798,12d,3.5,Eightfold.AI,Machine Learning Engineer,2016.0,Company - Private,Enterprise Software & Network Solutions,Information Technology,121.0,163.0,Santa Clara,CA,United States,501.0,1000.0,NaN,NaN
799,30d+,4.4,Adobe,Research Scientist/Engineer,1982.0,Company - Public,Computer Hardware Development,Information Technology,121.0,239.0,San Jose,CA,United States,10000.0,10000.0,5.000000e+09,1.000000e+10
800,30d+,4.5,Curai,Machine Learning Engineer (Remote),2017.0,Company - Private,Hospitals & Health Clinics,Healthcare,150.0,200.0,San Francisco,CA,United States,51.0,200.0,NaN,NaN


We divide the company's revenue into the maximum and minimum value.

In [10]:
df.isna().sum()

PublishedAgo      0
Rating            0
Employer          0
JobTitle          0
Founded          29
Type              0
Industry         26
Sector           26
MinSalary         0
MaxSalary         0
City              0
State             0
Country           0
MinSize           5
MaxSize           5
MinRevenue      232
MaxRevenue      232
dtype: int64

In [11]:
df.replace(-1, np.NaN, inplace=True)

df.dropna(subset=['MinSalary', 'MaxSalary', 'Founded', 'Industry', 'Sector', 'MinSize'], inplace=True)

df.isna().sum()

PublishedAgo      0
Rating            0
Employer          0
JobTitle          0
Founded           0
Type              0
Industry          0
Sector            0
MinSalary         0
MaxSalary         0
City              0
State             0
Country           0
MinSize           0
MaxSize           0
MinRevenue      112
MaxRevenue      112
dtype: int64

Since the field we are most interested in is the salary, and we do not want to fill in with invented values (even if they are approximations based on the rest of the fields), we are going to eliminate all those columns where the salaries are NaN

In [12]:
df.to_csv('data_cleaned.csv', index=False)